In [80]:
import pandas as pd
import numpy as np

Reading in the training and test data we well as the column name to column label mapping file created in `column_name.ipynb`
File `country_continent.csv` is a mapping of 2-letter country code to the continent it is in
File `europe_country.csv` is a mapping of 2-letter country codes for European countries (and Israel) to the region of Europe it is in.

In [92]:
codebook=pd.read_csv(r"C:\Users\Jack\Desktop\Project441\column_name_file_clean.csv")
train=pd.read_csv(r"C:\Users\Jack\Desktop\Project441\Data\train_clean.csv")
train1=pd.read_csv(r"C:\Users\Jack\Desktop\Project441\Data\train_clean.csv")
test=pd.read_csv(r"C:\Users\Jack\Desktop\Project441\Data\test_clean.csv",index_col=False)
test1=pd.read_csv(r"C:\Users\Jack\Desktop\Project441\Data\test_clean.csv",index_col=False)
cc=pd.read_csv(r"C:\Users\Jack\Desktop\Project441\country_continent.csv",index_col=False)
eu=pd.read_csv(r"C:\Users\Jack\Desktop\Project441\Data\europe_country.csv",encoding='latin-1')

C:\Users\Jack\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,4,6,10,12,18,24,35,56,57,65,85,98,99,100,105,124,125,126,127,128,130,131,132,133,166,171,172,228,229,230,240,256,257,258) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Jack\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,6,10,12,21,24,35,85,100,124,126,127,128,130,131,133,166,172,177,220,222,228,229,230,240,257,258) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


We define a function to merge our continent table to our training/test data on `v17: Country of birth`, this will remove country of birth as a factor and replace it with continent of birth. This function also merges our European region table to our training/test data on `cntry: Country`, this will remove country as a factor and replace it with the region of Europe. We then apply this funciton to out train and test data. 

In [93]:
def convert_to_regions(df):
    df=df.merge(cc,how='left',right_on='iso 3166 country',left_on='v17')
    df=df.drop(['iso 3166 country','v20','v154','v155','v25'],axis=1)
    df=df.merge(eu,how='left',right_on='cntry',left_on='cntry')
    #df=df.drop(['cntry'],axis=1)
    return(df)

In [94]:
def same_birth_fam(df):
    df['mom_dad_same']=1*(df['v161']==df['v78'])
    df.loc[df['v161'].isin([str(num) for num in np.arange(0, 100, 1).tolist()]),'mom_dad_same']=np.nan
    df.loc[df['v78'].isin([str(num) for num in np.arange(0, 100, 1).tolist()]),'mom_dad_same']=np.nan

    df['mom_same']=1*(df['v161']==df['v17'])
    df.loc[df['v161'].isin([str(num) for num in np.arange(0, 100, 1).tolist()]),'mom_same']=np.nan
    df.loc[df['v17'].isin([str(num) for num in np.arange(0, 100, 1).tolist()]),'mom_same']=np.nan

    df['dad_same']=1*(df['v78']==df['v17'])
    df.loc[df['v78'].isin([str(num) for num in np.arange(0, 100, 1).tolist()]),'dad_same']=np.nan
    df.loc[df['v17'].isin([str(num) for num in np.arange(0, 100, 1).tolist()]),'dad_same']=np.nan


    df=df.drop(['v17','v78','v161'],axis=1)
    return(df)

In [95]:
train=convert_to_regions(train)
test=convert_to_regions(test)

train=same_birth_fam(train)
test=same_birth_fam(test)

Month the survey ended (`v128`) is recorded as a numerical variable. It does not make much sense to do this as it is possible that November and December are similar in some respect, but December is equally close to January, but 12 is not as close to 1 as it is to 11. So we recorded this value as a factor level. 

In a similar way, religion at present (`v190`) and religion in the past (`v191`) are recorded as numerical values, but relgion does not have any inherent ordering so we convert these values to characters, and edit column `v190` to have 'Religion_now1' instead of 1 and `v191` to have 'Religion_past1' instead of 1, where 1 corresponds to Roman Catholic in both cases. The direct mapping of religion number to denomination is found in `codebook_long.txt`.

In [96]:
train['v128']='Month'+train['v128'].astype(str)
test['v128']='Month'+test['v128'].astype(str)


train['v190']='Religion_now'+train['v190'].astype(str)
test['v190']='Religion_now'+test['v190'].astype(str)
train['v191']='Religion_past'+train['v191'].astype(str)
test['v191']='Religion_past'+test['v191'].astype(str)


train['is_jewish']=1*((train['v4']==22000)|(train['v5']==22000))
test['is_jewish']=1*((test['v4']==22000)|(test['v5']==22000))

train['is_gypsy']=1*((train['v4']==14100)|(train['v5']==14100))
test['is_gypsy']=1*((test['v4']==14100)|(test['v5']==14100))

Many respondants do not provide an actual answer to many of the questions. `Not available,Refusal,No Answer,Don't know,No answer Not applicable` are examples of such cases. We decide to impute all of these, with the exception of `Not applicable`. To get arround this we will convert all instances of `Not applicable` to a placebholder value, -999.

In [27]:
for c in train.columns.drop('satisfied'):
    try:
        train.loc[train[c].isin(['Not applicable']),c]=-999
        
        ### Addition
        test.loc[test[c].isin(['Not applicable']),c]=-999
        ####
    except ValueError:
        print(c+" is not numeric")

For Example: Respondant 37017 has responce `Don't Know` for `v263: Year of birth of second person in household`. So we create column `v263given`  37017 will have value 1 37017 will now have value `NaN` for column `v263`. Respondents who did not have `Not available, Refusal, No Answer, Don't know, No answer` provided for `v263` will have 0 in `v263given` and the original value in `v263`.

We have decide to treat all these responses the same.

In [28]:
for c in train.columns.drop('satisfied'):
    train[c+'given']=train[c].isin(['Not available','Refusal',"No Answer","Don't know",'No answer'])*1
    train.loc[train[c].isin(['Not available','Refusal',"No Answer","Don't know",'No answer']),c]=np.nan
    test[c+'given']=test[c].isin(['Not available','Refusal',"No Answer","Don't know",'No answer'])*1
    test.loc[test[c].isin(['Not available','Refusal',"No Answer","Don't know",'No answer']),c]=np.nan

We suppose that the makeup of the people living with a person will impact their happiness. We create relevant variables by looking at the `nth person in household: relationship to respondent` series of variables (`v204` to `v215`) and combining with the `Gender of nth person in household` variables (`v86` to `97`). 

We create variables `g_2` to `g_13` which correspond to the gender, relationship code of the nth person (n between 2 and 13). For example a `11` for variable `g_4` corresponds to a male child as the 4th person in the household. We can then collect duplicates to see how many male children are in the household. 

We can also look at the `Year of birth of nth person in household` variables to get the age of these members.

In [29]:
def kids_adult_kids(x):
    x['g_2']=x['v90']+x['v208']
    x['g_3']=x['v91']+x['v209']
    x['g_4']=x['v92']+x['v210']
    x['g_5']=x['v93']+x['v211']
    x['g_6']=x['v94']+x['v212']
    x['g_7']=x['v95']+x['v213']
    x['g_8']=x['v96']+x['v214']
    x['g_9']=x['v97']+x['v215']
    x['g_10']=x['v86']+x['v204']
    x['g_11']=x['v87']+x['v205']
    x['g_12']=x['v88']+x['v206']
    x['g_13']=x['v89']+x['v207']
    
    for combo in ['11','21','12','22','13','23','14','24','15','25','16','26']:
        x['total'+combo]=1*pd.DataFrame(x[['g_2','g_3','g_4','g_5','g_6','g_7','g_8','g_9','g_10',
                                           'g_11','g_12','g_13']]==combo).sum(axis=1)
        
    rel=train[['v204','v205','v206','v207','v208','v209','v210','v211','v212','v213','v214','v215']].copy()
    age=train[['v259','v260','v261','v262','v263','v264','v265','v266','v267','v268','v269','v270']].copy()

    rel.columns=[10,11,12,13,2,3,4,5,6,7,8,9]
    age.columns=[10,11,12,13,2,3,4,5,6,7,8,9]

    age[age.isna()]=0
    age[age.isin(['Not available','Refusal',"Don't know",'No answer','No Answer','Not applicable',np.nan])]=0

    rel[rel.isna()]=0
    rel[age.isin(['Not available','Refusal',"Don't know",'No answer',"No Answer",'Not applicable',np.nan])]=0

    for c in [10,11,12,13,2,3,4,5,6,7,8,9]:
        try:
            age[c]=age[c].astype(float)
        except ValueError:
            print(c+" is not numeric")
        
    x['adult_children']=((1*((age<=1990)&(age>0)))*(1*(rel=="2"))).sum(axis=1)
    x['kids']=((1*(age>2000))).sum(axis=1)
    
    
    older=age.copy()
    younger=age.copy()
    older['v258']=x['v258'].astype(float)
    younger['v258']=x['v258'].astype(float)
    for c in [10,11,12,13,2,3,4,5,6,7,8,9]:
        younger[c]=1*(younger[c]>younger['v258'])
        older[c]=1*((older[c]<older['v258'])&(older[c]>0))
    
    
    x['older']=older[[10,11,12,13,2,3,4,5,6,7,8,9]].sum(axis=1)
    x['younger']=younger[[10,11,12,13,2,3,4,5,6,7,8,9]].sum(axis=1)

   
    x=x.drop(['g_2','g_3','g_4','g_5','g_6','g_7','g_8','g_9','g_10','g_11','g_12','g_13'],axis=1)
    
    return(x)

In [30]:
train=kids_adult_kids(train)
test=kids_adult_kids(test)

Values are sometimes recorded in two datatypes. for example we observe the unique values for `v10: Born in country`:

In [31]:
train['v10'].unique()

array([1, 2, '1', '2', nan], dtype=object)

We see 1 and '1' are both possible values, to correct for this we decide to change the string values to be numerics:

In [32]:
for c in train.columns.drop('satisfied'):
    try:
        train[c]=train[c].astype(float)
        
        ### Addition
        test[c]=test[c].astype(float)
        ####
    except ValueError:
        print(c+" is not numeric")

v128 is not numeric
v190 is not numeric
v191 is not numeric
InterviewDayOfWeek is not numeric
continent code is not numeric
EU_region is not numeric


We can now check again to make sure this issue has been corrected:

In [33]:
train['v10'].unique()

array([ 1.,  2., nan])

All variables which are not `float` do not need to be imputed as NA can be left as a value which will be converted into a dummy variable.

In [34]:
train.columns[train.dtypes!='float64']

Index(['v128', 'v190', 'v191', 'satisfied', 'InterviewDayOfWeek',
       'continent code', 'EU_region'],
      dtype='object')

Make a separate training set with only numeric values

In [35]:
num_col=train.columns[~train.columns.isin(train.columns[train.dtypes!='float64'])]
train_n=train[num_col]
train_n=train_n.drop(columns={'id'})

#### Addition
num_col=test.columns[~test.columns.isin(test.columns[test.dtypes!='float64'])]
test_n=test[num_col]
test_n=test_n.drop(columns={'id'})

In [36]:
train_n.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,total14,total24,total15,total25,total16,total26,adult_children,kids,older,younger
0,2.0,2.0,74.0,11010.0,-999.0,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,2.0,58.0,11010.0,-999.0,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6.0,2.0,47.0,11010.0,11010.0,2.0,2.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
3,10.0,2.0,22.0,11010.0,-999.0,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,24.0,11010.0,-999.0,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Using `Imputer` from `sklearn` we impute these numerical columns with the mean.

In [37]:
test_n.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,total14,total24,total15,total25,total16,total26,adult_children,kids,older,younger
0,0.0,1.0,63.0,15020.0,-999.0,2.0,2.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.0,2.0,44.0,21070.0,21070.0,1.0,2.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10.0,2.0,21.0,11010.0,11010.0,2.0,1.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
3,2.0,2.0,75.0,11010.0,-999.0,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,22.0,11010.0,11010.0,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
test_n.nunique()[test_n.nunique()==0]

Series([], dtype: int64)

We create our response for our training data, and remove this variable from our training data

In [39]:
y_train=train['satisfied']

# Imputation

Recalling our placeholder value for `Not Applicable` of -999, we must adjust for these when we wish to impute the values for `Not available,Refusal,No Answer,Don't know,No answer Not applicable` which were previously converted to `NaN`. We impute these values to be the median value of all other responses in that data set while omiting the -999 values. 

In [40]:
from statistics import mode

In [41]:
train_n.loc[train_n[c].isin([-999,np.nan]),c].mode()
test_n.loc[test_n[c].isin([-999,np.nan]),c].mode()

Series([], dtype: float64)

In [43]:
for c in train_n.columns:
    try:
        train_n.loc[train_n[c].isna(),c]=train_n.loc[~train_n[c].isna(),c].mean()
        
        ### Addition
        test_n.loc[test_n[c].isna(),c]=test_n.loc[~test_n[c].isna(),c].mean()
        ####
    except ValueError:
        print(c+" is not numeric")
        
## Addition
for v in ['v89','v195','v207','v262']:
    test_n[v]=(train_n[v]).mean()

Make a separate training set with only the character variables:

In [44]:
train_c=train[train.columns[(train.dtypes!='float64')&(train.columns!='satisfied')]]

## Addition
test_c=test[test.columns[(test.dtypes!='float64')&(test.columns!='satisfied')]]

For these character variables, we convert them to dummy variables where each level will have it's own variable with levels:
- 0 if the respondent did not give that level 
- 1 if the respondent gave that level 

In [45]:
train_c=pd.get_dummies(train_c)

## Addition
test_c=pd.get_dummies(test_c)

In [46]:
for v in train_c.columns[~train_c.columns.isin(test_c.columns)]:
    print(v)

We now append all these dummy variable columns to our `train_n` dataframe to create a finalized training set, with no missing data in it. 

In [47]:
for c in train_c.columns:
    train_n[c]=train_c[c]
    
    ## Addition
    test_n[c]=test_c[c]

## Addition
for v in train_c.columns[~train_c.columns.isin(test_c.columns)]:
    test_n[v]=0


Many Variables can be dropped for computational easy, especially numeric variables which do not preserve ordering. 

For Example:

- v4: First ancestry, European Standard Classification of Cultural and Ethnic Groups
- v5: Second ancestry, European Standard Classification of Cultural and Ethnic Groups
- v86 to v97: Gender of nth person in house 
- v124 to  v127, and v129: End/Start of interview month, minute hour not changing v128 since this variable was already removed and converted to factor

- v192 to v203: Relationship to respondent of nth person in house
- v204 to v215: Relationship to respondent of nth person in house
- v258 : Year of Birth
- v259 to v270: Year of Birth of nth person in house






In [48]:
v=['v270','v269','v268','v267','v266','v265','v264','v263','v262','v261','v260','v259','v258','v215','v214','v213','v212','v211',
   'v210','v209','v208','v207','v206','v205','v204','v203','v202','v201','v200','v199','v198','v197','v196','v195','v194','v193',
   'v192','v129','v127','v126','v125','v124','v97','v96','v95','v94','v93','v92','v91','v90','v89','v88','v87','v86','v5','v4']

for c in v:
    train_n=train_n.drop(c,axis=1)
    test_n=test_n.drop(c,axis=1)

In [49]:
train_n.head()

,v1,v2,v3,v6,v7,v8,v9,v10,v11,v12,...,continent code_AS,continent code_EU,continent code_OC,continent code_SA,EU_region_British_Isles_Europe,EU_region_East_Europe,EU_region_North_Europe,EU_region_Other_Europe,EU_region_South_Europe,EU_region_West_Europe
0,2.0,2.0,74.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,...,0,0,0,0,0,0,0,0,0,1
1,4.0,2.0,58.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,1
2,6.0,2.0,47.0,2.0,2.0,1.0,2.0,1.0,-999.0,1.0,...,0,0,0,0,0,0,0,0,0,1
3,10.0,2.0,22.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,1
4,0.0,1.0,24.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,...,0,0,0,0,0,0,0,0,0,1


# Break

In [50]:
train_n['v150']=train_n['v150']//1000
test_n['v150']=test_n['v150']//1000

train_n['v151']=train_n['v151']//1000
test_n['v151']=test_n['v151']//1000

train_n['better_job_partner']=train_n['v150']>train_n['v151']
test_n['better_job_partner']=test_n['v150']>test_n['v151']

train_n['equal_job_partner']=1*(train_n['v150']==train_n['v151'])
test_n['equal_job_partner']=1*(test_n['v150']==test_n['v151'])

In [51]:
train_n=train_n[train_n.columns[~train_n.columns.isin(['v131'])]]
test_n=test_n[test_n.columns[~test_n.columns.isin(['v131'])]]

In [52]:
train_n['satisfied']=y_train

In [53]:
train_n.to_csv(r"C:\Users\Jack\Desktop\Project441\train_n.csv",index = False)
test_n.to_csv(r"C:\Users\Jack\Desktop\Project441\test_n.csv",index = False)